### **przyjazny bot**
#### post lyrics from top 50 songs in Poland on Twitter

Import necessary libraries

In [1]:
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
import tweepy
import random
import os
import pandas as pd
from dotenv import load_dotenv
import requests
import urllib.request
import re

##### Spotify API Authorization

In [2]:
load_dotenv()

auth_manager = SpotifyClientCredentials(client_id=os.getenv("SPOTIPY_CLIENT_ID"), client_secret=os.getenv("SPOTIPY_CLIENT_SECRET"))
sp = spotipy.Spotify(auth_manager=auth_manager)

Download track ids from specific playlist

In [3]:
def download_tracks_ids(username, playlist_id):
    tracks_ids = []
    playlist = sp.user_playlist(username, playlist_id)
    for item in playlist['tracks']['items']:
        track = item['track']
        tracks_ids.append(track['id'])
    return tracks_ids

In [4]:
#tracks_ids = download_tracks_ids(os.getenv("USERNAME_2"), os.getenv("PLAYLIST_ID")) 
tracks_ids = download_tracks_ids(os.getenv("USERNAME"), os.getenv("PLAYLIST_ID")) 
print(tracks_ids)

['0yLdNVWF3Srea0uzk55zFn', '05MYJf19LSo2E1lZ0RrMAC', '3Nqxx5fdl3FQlrk3zarRmA', '09m29we7x9tgxyo6f3OQ87', '1pCekfmD7qmOpKnTgttGcu', '384lHduuDsrR4HPUwR3fG4', '4JmaOG7bfj1uNv8jEUg0Ce', '4nrPB8O7Y7wsOCJdgXkthe', '3f9Ugg030LDe0p6ospTORC', '3nqQXoyQOWXiESFLlDF1hG', '73vIOb4Q7YN6HeJTbscRx5', '3T601uaEcVdVmGE5w3BSDW', '4uUG5RXrOk84mYEfFvj3cK', '3p6uAP2ajkobkIW8IkEpgr', '08sMDBcjUBXWV99gZI092C', '2yxPqxHbj1L7vrCpHLtAcG', '1Qrg8KqiBpW07V7PNxwwwL', '4N6rgD6EqLbWcl5ld3NqTP', '5GdFPcbGDQqtbVO2jl6Ez5', '1yiGGRGdHEc1F3jF5jEVmH', '4nv5o5Xo4ySBaIVnXr75Xs', '561jH07mF1jHuk7KlaeF0s', '11BKm0j4eYoCPPpCONAVwA', '0zmxM4MXfisJRTQcPa1wbv', '4lPAZiTJbaQ4irMzcq0GOU', '7jtQIBanIiJOMS6RyCx6jZ', '2dHHgzDwk4BJdRwy9uXhTO', '0JXXNGljqupsJaZsgSbMZV', '2yEWVnp65tTXSrRW03IqXs', '53OQFPwyflA0jMyUjUSawr', '1VoUHMVxu1qG2Z9josOhWs', '4h9wh7iOZ0GGn8QVp4RAOB', '14154tJOTjeyeOyhnqP4QM', '5WxVXxCMRnvxUKFq40ELwq', '1N8TTK1Uoy7UvQNUazfUt5', '11JXHNIH2xO5INcJQOWCjY', '2KTfSSKBBfMLXwDqrtoOE5', '6JTuuvDOmXEgBdPNxG3tV3', '2nGA88iGan

Draw of a track id

In [5]:
track_id = random.choice(tracks_ids)
print(track_id)

561jH07mF1jHuk7KlaeF0s


Collect data (title and main artist) needed to search lyrics via the Genius API

In [6]:
def artist_genius(track_id):
    track_info = sp.track(track_id)
    artist_genius = track_info['album']['artists'][0]['name']
    return artist_genius

In [7]:
def title(track_id):
    song_info = sp.track(track_id)
    title = song_info['name']
    return title

Collect information about all artists to include in a tweet

In [8]:
def artists(track_id):
    track_info = sp.track(track_id)
    artists_list = []
    for i in range(0, len(track_info['album']['artists'])):
        artists_list.append(track_info['album']['artists'][i]['name']) 
    str_artists = ", "
    return(str_artists.join(artists_list))

In [9]:
print(artist_genius(track_id), title(track_id), artists(track_id))

Eminem Mockingbird Eminem


Get album cover for given song

In [10]:
def get_album_cover(track_id):
    track_info = sp.track(track_id = track_id)
    img_url = track_info['album']['images'][0]['url']
    img_data = requests.get(img_url).content
    with open('album_cover.jpg', 'wb') as handler:
        handler.write(img_data)

In [11]:
get_album_cover(track_id)
print(sp.track(track_id = track_id)['album']['images'][0]['url'])

https://i.scdn.co/image/ab67616d0000b273726d48d93d02e1271774f023


Get related artists for given artist

In [12]:
def get_related_artists(artist_name): # it's typo sensitive
    results = sp.search(q='artist:' + artist_name, type='artist')
    if results['artists']['total'] == 0: 
        return "Couldn't find artist"
    else: 
        artist_id = results['artists']['items'][0]['id']
        related_artists = sp.artist_related_artists(artist_id)['artists']
        related_artists_list = []
        for artist in related_artists[:5]:
            related_artists_list.append(artist['name'])
        return related_artists_list

In [13]:
def tweet_content2(artists):
    tweet = 'Here is the list of similar artists:' + '\n' + '\n' + '1. ' + artists[0] + '\n' + '2. ' + artists[1] + '\n' + '3. ' + artists[2] + '\n' + '4. ' + artists[3] + '\n' + '5. ' + artists[4]
    return tweet

In [14]:
tweet_content3 = "Sorry, I couldn't find this artist. Are you sure you spelled it right?"

Get link of teledisc on YouTube

In [15]:
def prepare_text_to_link(string):
    splited = string.split()
    name = ""
    iterator = 0
    for i in splited:
        iterator = iterator+1
        if iterator==1:
            name = i
        else:
            name = name+"+"+i
    return name

In [16]:
def get_body_for_youtube(track_id):
    title_name = title(track_id)
    artists_name = artists(track_id)
    body = artists_name + title_name
    return body

In [17]:
def get_youtube_link(body):
    body = body.encode('ascii', 'ignore').decode('ascii')
    html = urllib.request.urlopen("https://www.youtube.com/results?search_query=" + body)
    video_id = re.findall(r"watch\?v=(\S{11})",html.read().decode())
    link = "https://www.youtube.com/watch?v=" + video_id[0]
    return link
    

In [18]:
def tweet_content_yt():
    tweet = 'Here is the link to your teledisc. Enjoy! ' + "\n" + "\n" + link
    return tweet

##### Genius API

In [19]:
genius = lyricsgenius.Genius(os.getenv("GENIUS_CLIENT_ID"))
genius.remove_section_headers = True

Download track lyrics from Genius

In [20]:
def raw_lyrics(track_id):
    lyrics = genius.search_song(title(track_id), artist_genius(track_id)).lyrics
    return lyrics

Processing the text to include it in the tweet

In [21]:
def tweet_content(lyrics, track_id):
    lyrics = lyrics.split('\n')
    for index in range(len(lyrics)):
        if lyrics[index] == "" or "[" in lyrics[index]:
            lyrics[index] = "XXX"
    lyrics = [i for i in lyrics if i != "XXX"]

    random_num = random.randrange(0, len(lyrics)-1)
    tweet = lyrics[random_num] + "\n" + lyrics[random_num+1] + "\n" + "\n" + title(track_id) + " by " + artists(track_id)
    tweet = tweet.replace("\\", "")
    return tweet

In [22]:
lyrics = raw_lyrics(track_id)
tweet_content(lyrics, track_id)

Searching for "Mockingbird" by Eminem...
Done.


"But no more worries, rest your head and go to sleep\nMaybe one day we'll wake up and this'll all just be a dream\n\nMockingbird by Eminem"

##### Twitter API

First API v1.1 authentication, only to use *media_upload* method (it's not available in API v2)

In [23]:
auth = tweepy.OAuthHandler(os.getenv("CONSUMER_KEY"), os.getenv("CONSUMER_SECRET"))
auth.set_access_token(os.getenv("ACCESS_TOKEN"), os.getenv("ACCESS_TOKEN_SECRET"))

apiv1 = tweepy.API(auth)

Upload album cover to twitter (we have to do that in order to attach it to tweet later)

In [24]:
media = apiv1.media_upload(filename = "album_cover.jpg")
media_id = media.media_id_string #media_id potem trzeba wstawić do metody create_tweet

And now API v2 authentication

In [25]:
api = tweepy.Client(bearer_token=os.getenv("BEARER_TOKEN"), consumer_key= os.getenv("CONSUMER_KEY"), consumer_secret=os.getenv("CONSUMER_SECRET"), access_token=os.getenv("ACCESS_TOKEN"), access_token_secret=os.getenv("ACCESS_TOKEN_SECRET"))

Tweet lyrics of one of TOP 50 tracks in Poland

In [26]:
result = None
while result is None:
    try:
        lyrics = raw_lyrics(track_id)
        result = api.create_tweet(text=tweet_content(lyrics, track_id), media_ids = [media_id])
    except:
         pass

Searching for "Mockingbird" by Eminem...
Done.


#### Retweet quote tweets and reply on mention

Draw of a track id

In [27]:
track_id = random.choice(tracks_ids)
print(track_id)

4JmaOG7bfj1uNv8jEUg0Ce


Prepare string to put into YouTube link

In [28]:
get_body_for_youtube(track_id)
body = prepare_text_to_link(get_body_for_youtube(track_id))
print(body)

Kuban,+Favstna+okrągło


Get Youtube link for chosen song

In [29]:
get_body_for_youtube(track_id)
body = prepare_text_to_link(get_body_for_youtube(track_id))
#print(body)
link = get_youtube_link(body)
print(link)

https://www.youtube.com/watch?v=DIgIjGDtmdg


In [30]:
def tweet_content_yt():
    tweet = 'Here is the link to your teledisc. Enjoy! ' + "\n" + "\n" + link
    return tweet

Create content for post with YouTube link

In [31]:
tweet_content_yt()

'Here is the link to your teledisc. Enjoy! \n\nhttps://www.youtube.com/watch?v=DIgIjGDtmdg'

In [32]:
def get_tweet(id):
    tweet = api.get_tweet(id, expansions=['author_id'], user_fields=['username'])
    return tweet

In [33]:
def quote_tweets(query, max_results):
    tweets = api.search_recent_tweets(query=query, max_results=max_results)
    results = []
    if not tweets.data is None and len(tweets.data) > 0:
        for tweet in tweets.data:
            twt = get_tweet(tweet['id'])
            obj = {}
            obj['id'] = tweet.id
            obj['username'] = twt.includes['users'][0].username
            obj['text'] = tweet.text
            results.append(obj)
    return results

Check if whether a tweet (id) has already received a reply

In [34]:
def check(id):
    with open('last_tweet_ids.txt') as f:
        datafile = f.readlines()
    for line in datafile:
        if id in line:
            return True
    return False

Add tweet (id) to database = text file


In [35]:
def append_new_id(text_to_append):
    with open('last_tweet_ids.txt', "a+") as file_object:
        file_object.seek(0)
        data = file_object.read(100)
        if len(data) > 0:
            file_object.write("\n")
        file_object.write(text_to_append)

Get artist name from the tweet

In [36]:
def get_artist_name(tweet_text):
    start = '@przyjaznybot'
    end = 'related artists'
    artist_name = tweet_text[tweet_text.find(start)+len(start):tweet_text.rfind(end)]
    return artist_name

In [37]:
QUERY = "przyjaznybot"
MAX_RESULTS = 10

quote_tweets(QUERY, MAX_RESULTS)

[{'id': 1618648089752920064,
  'username': 'przyjaznybot',
  'text': 'Papa was a rolling stone, Momma developed a habit\nAnd it all happened too fast for either one of us to grab it\n\nMockingbird by Eminem https://t.co/YYJ6pKJX42'},
 {'id': 1618646440754188290,
  'username': 'Szafa9',
  'text': '@przyjaznybot kino related artists'},
 {'id': 1618643953053753346,
  'username': 'Szafa9',
  'text': '@przyjaznybot the smiths related artists'},
 {'id': 1618642948677144576,
  'username': 'przyjaznybot',
  'text': 'hello @Szafa9\n\nJeszcze kilka słów\nA jeśli już, to wezmę tam ze sobą bukiet róż\n\nmori by Dawid Podsiadło https://t.co/pKpxsnTfSv'},
 {'id': 1618642945166528519,
  'username': 'przyjaznybot',
  'text': 'hello @Szafa9\n\nmori by Dawid Podsiadło. Here is the link to your teledisc. Enjoy! \n\nhttps://t.co/4VFqDcbAob https://t.co/vEA9Qh0bos'},
 {'id': 1618642260266213378,
  'username': 'Szafa9',
  'text': '@przyjaznybot send top Poland music link xyz'},
 {'id': 1618642145321320448,


From list of dictionaries to list of specific variables

In [38]:
values_of_usernames = [dict['username'] for dict in quote_tweets(QUERY, MAX_RESULTS)]
values_of_ids = [dict['id'] for dict in quote_tweets(QUERY, MAX_RESULTS)]
values_of_text = [dict['text'] for dict in quote_tweets(QUERY, MAX_RESULTS)]

If someone share a tweet containing the phrase "draw lyrics", bot create new tweet with the user's mention. <br>

If someone share a tweet containing the phrase "send top Poland music link", bot create new tweet with the user's mention, <br>
photo of the album and link to the song on YouTube <br>

If someone share a tweet containing the phrase "*artist* related artists", bot replies with list of related artists. <br>

If someone quotes a tweet, bot retweet it.

In [39]:
lines = []
lyrics = raw_lyrics(track_id)

get_album_cover(track_id)
media = apiv1.media_upload(filename = "album_cover.jpg")
media_id = media.media_id_string

for i in range(len(values_of_ids)):
    if check(str(values_of_ids[i])) == False:
        if values_of_usernames[i] != "przyjaznybot":
            if "draw lyrics" in values_of_text[i] and check(str(values_of_ids[i])) == False:
                reply = "hello @" + values_of_usernames[i] + "\n" + "\n" + tweet_content(lyrics, track_id)
                api.like(values_of_ids[i])
                api.create_tweet(text=reply, media_ids = [media_id])
                append_new_id(str(values_of_ids[i]))
            elif "send top Poland music link" in values_of_text[i] and check(str(values_of_ids[i])) == False:
                reply = "hello @" + values_of_usernames[i] + "\n" + "\n" + title(track_id) + " by " + artists(track_id) + ". " + tweet_content_yt()
                api.like(values_of_ids[i])
                api.create_tweet(text=reply, media_ids = [media_id])
                append_new_id(str(values_of_ids[i]))
            elif "related artists" in values_of_text[i] and check(str(values_of_ids[i])) == False:
                artist_name = get_artist_name(values_of_text[i])
                related_artists = get_related_artists(artist_name)
                if related_artists == "Couldn't find artist":
                    reply = "hello @" + values_of_usernames[i] + "\n" + "\n" + tweet_content3
                else:
                    reply = "hello @" + values_of_usernames[i] + "\n" + "\n" + tweet_content2(related_artists)
                api.like(values_of_ids[i])
                api.create_tweet(text = reply, in_reply_to_tweet_id = values_of_ids[i])
                append_new_id(str(values_of_ids[i]))
            else:
                api.like(values_of_ids[i])
                api.retweet(values_of_ids[i])
                append_new_id(str(values_of_ids[i]))

Searching for "na okrągło" by Kuban...
Done.
